# 複数のデータフレームの結合

複数のデータフレームを結合する操作は、データの統合や分析において非常に重要な手段です。Polarsでは、さまざまな結合方法が提供されており、データの構造に応じて最適な方法を選択することができます。以下では、Polarsで利用できる代表的な結合手法である`concat`、`join`、`join_asof`、`join_where`とデータフレームの結合メソッドについて紹介します。

In [2]:
import polars as pl
from helper.jupyter import row

## concat

`pl.concat()`を使用すると、複数のデータフレームを縦または横に結合できます。結合方法は引数`how`で指定され、以下の5種類の結合方法があります。

* `vertical`および`vertical_relaxed`: 縦方向の結合
* `horizontal`: 横方向の結合
* `diagonal`および`diagonal_relaxed`: 縦横両方向の結合
* `align`: 縦横両方向の結合ですが、データをキーで整列してから結合を行います

以下は、次の二つのデータフレームを使って、上記の結合方法について詳しく説明します。

In [54]:
df1 = pl.DataFrame({"x":[1, 2, 3], "y":[2, 3, 1]})
df2 = pl.DataFrame({"x":[6, 2, 1, 5], "y":[12, 3, 2, 4]})
row(df1, df2)

x,y
i64,i64
1,2
2,3
3,1
x,y
i64,i64
6,12
2,3
1,2
5,4


### 縦結合

以下のように、すべてのデータフレームの列名とデータ型が一致する場合は、`vertical`で縦に結合します。

In [41]:
pl.concat([df1, df2], how='vertical')

x,y
i64,i64
1,2
2,3
3,1
6,12
2,3
1,2
5,4


列名が一致するがデータ型が一致しない場合は、`vertical_relaxed`を使用して縦に結合します。この場合、結果のデータ型は上位のデータ型が採用されます。以下のコード例では、`df2`の`x`列を`Float64`型にキャストしてから結合しています。このように、`x`列のデータ型が`Float64`に統一され、縦に結合されます。

In [32]:
pl.concat([
    df1, 
    df2.with_columns(pl.col('x').cast(pl.Float64))
    ], 
    how='vertical_relaxed')

x,y
f64,i64
1.0,2
2.0,3
3.0,1
6.0,12
2.0,3
1.0,2
5.0,4


Pandasのように縦結合するとき、各データフレームにキーを付ける方法についてのプログラム例を以下に示します。この方法では、`df1`と`df2`にそれぞれキーを付けてから縦に結合します。
プログラムには、`key`列を追加して各データフレームの行にキーを付けてから縦に結合することで、元のデータフレームを識別できるようにしています。

In [46]:
data = {"A":df1, "B":df2}
pl.concat([
    df.select(pl.lit(key).alias("key"), pl.all()) 
    for key, df in data.items()
])

key,x,y
str,i64,i64
"""A""",1,2
"""A""",2,3
"""A""",3,1
"""B""",6,12
"""B""",2,3
"""B""",1,2
"""B""",5,4


### 横結合

列名が異なるデータフレームを横に結合するには、`horizontal`を使用します。以下のプログラムでは、`df1`と`df2`の列名を2種類の方法でリネームし、横結合します。このように、`df1`の列名に`1`を、`df2`の列名に`2`を付けて横に結合します。`df2`は`df1`より行数が多いため、`df1`に存在しない行には`null`が補完されます。

In [33]:
pl.concat([
    df1.rename(lambda name:f"{name}1"), 
    df2.select(pl.all().name.suffix("2"))
], how='horizontal')

x1,y1,x2,y2
i64,i64,i64,i64
1,2,6,12
2,3,2,3
3,1,1,2
null,null,5,4


一部の列名が同じで、一部の列名が異なる場合、`diagonal`や`diagonal_relaxed`を使用して結合できます。`diagonal_relaxed`は自動的に上位のデータ型を採用します。次のプログラムでは、`df1`に`u`列が追加され、`df2`に`v`列が追加され、`diagonal`で二つのデータフレームを結合します。列名が一致するデータは縦に結合し、一致しない列は`NULL`で欠損値を表します。

### 縦と横結合

In [36]:
dfs = [
    df1.with_columns(u=pl.col('x') + pl.col('y')),
    df2.with_columns(v=pl.col('x') * pl.col('y'))
]
pl.concat(dfs, how='diagonal')

x,y,u,v
i64,i64,i64,i64
1,2,3,null
2,3,5,null
3,1,4,null
6,12,null,72
2,3,null,6
1,2,null,2
5,4,null,20


### 整列結合

`align`結合は、`diagonal`と似ていますが、列名が一致するデータをキーとして集合化し、他の列の値を統合します。以下は、`df1`と`df2`に追加した列を使って`align`で結合する例です。このように、`align`結合では共通の`x`と`y`の値をキーとして行をマージし、他の列の値を統合しています。例えば、`x=1, y=2`の行は`u=3`と`v=2`が統合されて1行になります。

In [37]:
pl.concat(dfs, how="align")

x,y,u,v
i64,i64,i64,i64
1,2,3,2
2,3,5,6
3,1,4,null
5,4,null,20
6,12,null,72


`pl.align_frames()`を使用すると、複数のデータフレームを指定した列で整列させることができます。以下の例では、`df1`と`df2`を`x`および`y`列で整列させています。整列後の各データフレームの行数は同じで、指定された列の値に基づいて他の列が整列されています。

In [53]:
row(*pl.align_frames(*dfs, on=['x', 'y']))

x,y,u
i64,i64,i64
1,2,3
2,3,5
3,1,4
5,4,null
6,12,null
x,y,v
i64,i64,i64
1,2,2
2,3,6


## join

`Polars`の`join()`メソッドは、SQLのように2つのデータフレームを結合するための方法を提供します。`join`は、異なる結合戦略を使用して、2つのデータフレームの対応する行をマッチさせることができます。

```python
df.join(
    other,                # 結合するもう1つのDataFrame
    on=None,              # 両方のDataFrameの結合に使う列名または式
    how='inner',          # 結合方法（デフォルトは'inner'）
    left_on=None,         # 左側のDataFrameの結合列
    right_on=None,        # 右側のDataFrameの結合列
    suffix='_right',      # 重複した列名に付ける接尾辞
    validate='m:m',       # 結合タイプの検証 ('m:m', 'm:1', '1:m', '1:1')
    join_nulls=False,     # Null値もマッチさせるかどうか
    coalesce=None         # 共通のキー列に対してnull値を埋めるかどうか
)
```

引数`how`で結合方法を指定します。

- **inner**: 両方のテーブルで一致する行を返す。
- **left**: 左のテーブルのすべての行と、右のテーブルの一致する行を返す。
- **right**: 右のテーブルのすべての行と、左のテーブルの一致する行を返す。
- **full**: 左右どちらかに一致する行をすべて返す。
- **semi**: 左テーブルから一致する行を返すが、右のテーブルからは列を返さない。
- **anti**: 左テーブルの一致しない行を返す。

In [2]:
df_left = pl.DataFrame({
    "id": [1, 2, 3, 4],
    "name": ["Alice", "Bob", "Charlie", "David"]
})

df_right = pl.DataFrame({
    "id": [3, 4, 5],
    "age": [23, 30, 40]
})

### inner

両方のデータフレームに存在する`id`に基づいて、内部結合を行います。次の例では、`id`が3と4に一致する行のみが返されました。

In [3]:
df_left.join(df_right, on="id", how="inner")

id,name,age
i64,str,i64
3,"""Charlie""",23
4,"""David""",30


### leftとright

左のデータフレームのすべての行を返し、右のデータフレームに一致するデータがあれば、それも含めます。次の例では、`id`が1と2の行は右に対応するデータがないため、`age`は`null`です。

In [4]:
df_left.join(df_right, on="id", how="left")

id,name,age
i64,str,i64
1,"""Alice""",null
2,"""Bob""",null
3,"""Charlie""",23
4,"""David""",30


In [5]:
df_left.join(df_right, on="id", how="right")

name,id,age
str,i64,i64
"""Charlie""",3,23
"""David""",4,30
null,5,40


### full

両方のデータフレームのすべての行を返し、どちらかに存在するデータがあれば、それを含めます。左と右のどちらからデータを取得したかを区別するために、結果には二つの結合列が作成されます。右側の結合列には、重複を避けるために`_right`という接尾辞が追加されます。結果から、idが1と2の行は左側のデータにのみ存在し、idが5の行は右側のデータにのみ存在することがわかります。

In [6]:
df_left.join(df_right, on="id", how="full")

id,name,id_right,age
i64,str,i64,i64
1,"""Alice""",null,null
2,"""Bob""",null,null
3,"""Charlie""",3,23
4,"""David""",4,30
null,null,5,40


`coalesce`引数を`True`に設定すると、これらの2つの列は1つにまとめられます。

In [7]:
df_left.join(df_right, on="id", how="full", coalesce=True)

id,name,age
i64,str,i64
1,"""Alice""",null
2,"""Bob""",null
3,"""Charlie""",23
4,"""David""",30
5,null,40


### semiとanti

`semi`は右側に存在する行を出力します。`anti`は右側に存在しない行を出力します。semiとantiの結果には、右側の列は含まれません。

In [10]:
df_left.join(df_right, on="id", how="semi")

id,name
i64,str
3,"""Charlie"""
4,"""David"""


In [11]:
df_left.join(df_right, on="id", how="anti")

id,name
i64,str
1,"""Alice"""
2,"""Bob"""


### cross

`cross`は、2つのデータフレームのデカルト積を出力します。つまり、左側のすべての行と右側のすべての行の組み合わせを結合します。

In [13]:
df_left.join(df_right, on="id", how="cross")

id,name,id_right,age
i64,str,i64,i64
1,"""Alice""",3,23
1,"""Alice""",4,30
1,"""Alice""",5,40
2,"""Bob""",3,23
2,"""Bob""",4,30
…,…,…,…
3,"""Charlie""",4,30
3,"""Charlie""",5,40
4,"""David""",3,23


## update

`DataFrame.update`メソッドは、Polarsにおいてインデックス列を基に既存のデータフレームを更新するために使用されます。引数`on`で指定された列が一致する行に対して、他の列のデータを更新します。

以下のコードは、`df`と`df_update`の`id`列が一致する行を基に、`df_update`の他の列（`x`と`y`）の値を`df`に更新する例です。

In [5]:
df = pl.DataFrame({
    "id": [1, 2, 3, 4],
    "x": [10, 20, 30, 40],
    "y": [40, 30, 20, 10]
})

df_update = pl.DataFrame({
    "id": [1, 3],
    "x": [100, 300],
    "y": [-10, -20],
})

df_res = df.update(df_update, on='id')
row(df, df_update, df_res)

id,x,y
i64,i64,i64
1,10,40
2,20,30
3,30,20
4,40,10
id,x,y
i64,i64,i64
1,100,-10
3,300,-20
id,x,y


## join_asof

`join_asof`は、時間や数値のような連続的なデータに基づいて2つのDataFrameを「概ね一致」させて結合するメソッドです。これは、正確な一致ではなく、片方の値がもう片方の値の近くにある場合に使われます。主に、**時系列データ**のような順序のあるデータで利用されます。

`join_asof`は通常、次のような状況で使われます：
- 片方のデータが特定の時間に対するスナップショットを持ち、もう片方がその時間に最も近い値を持っている場合。
- "前方一致"または"後方一致"など、指定された方向に最も近いデータを探す場合。

In [1]:
import polars as pl

df1 = pl.DataFrame(
    {
        "time": [1, 5, 10, 15, 20],
        "event": ["A", "B", "C", "D", "E"],
    }
)

df2 = pl.DataFrame(
    {
        "time": [2, 6, 12, 18],
        "price": [100, 105, 110, 115],
    }
)

result = df1.join_asof(df2, on="time", strategy="backward")
result

time,event,price
i64,str,i64
1,"""A""",null
5,"""B""",100
10,"""C""",105
15,"""D""",110
20,"""E""",115


`df1`の各行に対して、`df2`の`"time"`列で最も近くて「過去または現在の時間」にあたる行を結合します。つまり、`df1`の各行に対して、`df2`でその`"time"`に一番近い過去の`"price"`の値を結合します。`strategy="backward"`は、`df1`の`"time"`の値に対して、それよりも過去または同時刻の`df2`の値を選ぶという戦略です。もう一つのオプションに`"forward"`があり、これは未来の値を選択します。

## join_where

`join_where`では二つのDataFrameの列同士の比較条件を指定して、それに基づいて結合を行います。例えば、以下のコードでは、`df1`のtime列の値が、`df2`のtime_span列に含まれる時間範囲内にある場合に結合が行われます。一つの条件式には一つの比較演算子しか使用できません。複数の条件式がある場合は、それらすべての条件式を満たす場合にのみ結合が行われます。

In [8]:
df1 = pl.DataFrame(
    {
        "id": [100, 101, 102],
        "time": [120, 140, 160],
    }
)
df2 = pl.DataFrame(
    {
        "t_id": [404, 498, 676, 742],
        "time_span": [(100, 110), (110, 130), (90, 100), (150, 170)],
    }
)
df1.join_where(
    df2,
    pl.col('time') >= pl.col('time_span').list.get(0),
    pl.col('time') <= pl.col('time_span').list.get(1)
)

id,time,t_id,time_span
i64,i64,i64,list[i64]
102,160,742,"[150, 170]"
100,120,498,"[110, 130]"


## extendとvstackメソッド

`DataFrame.extend()` と `DataFrame.vstack()` は、既存のデータフレームに別のデータフレームの内容を行として追加するメソッドです。

- **`extend()`**: 元のデータフレームを直接変更します（その場で変更）。変更後のデータフレームは返り値として返します。
- **`vstack()`**: `in_place` 引数を指定することで、元のデータフレームを直接変更する（その場で変更）か、新しいデータフレームを返すかを選べます。

In [14]:
df1 = pl.DataFrame({"a": [1, 2], "b": [3, 4]})
df2 = pl.DataFrame({"a": [5, 6], "b": [7, 8]})

# clone()を使って元のデータフレームを複製
df_extend = df1.clone()
df_extend.extend(df2)  # extendで直接変更

df_vstack = df1.clone()
df_vstack.vstack(df2, in_place=True)  # vstackでその場で変更
row(df_extend, df_vstack)

a,b
i64,i64
1,3
2,4
5,7
6,8
a,b
i64,i64
1,3
2,4
5,7


両者の違いは、メモリ管理の方法にあります。

`extend()` は、引数のデータフレームの内容を元のデータフレームのメモリの末尾にコピーします。結果として、結合後のデータフレームのメモリは連続しています。ただし、元のデータフレームのメモリが再配置される可能性があるため、ループ内で頻繁に `extend()` を使用することは推奨されません。

`vstack()` は、結合する2つのデータフレームのメモリをそのままにし、リンクによって両者を結合します。そのため、結果として得られるデータフレームのメモリは不連続になります。この不連続性により、後で計算を行う際のパフォーマンスが低下する可能性があります。  
一般的には、ループ内で `vstack()` を使用してデータを結合し、最後に `rechunk()` を実行してメモリを連続した状態に変換するのが効率的です。

In [15]:
print(f'{df_extend.n_chunks() = }')           # extend後のチャンク数
print(f'{df_vstack.n_chunks() = }')          # vstack後のチャンク数
print(f'{df_vstack.rechunk().n_chunks() = }')  # rechunk後のチャンク数

df_extend.n_chunks() = 1
df_vstack.n_chunks() = 2
df_vstack.rechunk().n_chunks() = 1


## hstackメソッド

`DataFrame.hstack()` は、2つのデータフレームを水平方向（列方向）に結合します。`in_place=True` を指定すると、元のデータフレームに列を追加します。

In [18]:
df1 = pl.DataFrame({"a": [1, 2], "b": [3, 4]})
df2 = pl.DataFrame({"c": [5, 6], "d": [7, 8]})

# in-place操作なし（新しいデータフレームを返す）
df3 = df1.hstack(df2)

# in-place操作あり
df1.hstack(df2, in_place=True)
row(df3, df1)

a,b,c,d
i64,i64,i64,i64
1,3,5,7
2,4,6,8
a,b,c,d
i64,i64,i64,i64
1,3,5,7
2,4,6,8


## merge_sortedメソッド

`DataFrame.merge_sorted()` は、**あらかじめソートされた** 2つのデータフレームを縦方向に結合し、指定したキー列に基づいてソートされた新しいデータフレームを返すメソッドです。このメソッドを使用する際は、入力となる両方のデータフレームが事前に指定したキー列でソートされていることが前提となります。

In [26]:
df1 = pl.DataFrame({"a": [3, 1, 5], "b": [6, 4, 2]})
df2 = pl.DataFrame({"a": [6, 2, 4], "b": [7, 5, 3]})

# "a" 列でソートしてからマージ
df3 = df1.sort("a").merge_sorted(df2.sort("a"), key="a")

# "b" 列でソートしてからマージ
df4 = df1.sort("b").merge_sorted(df2.sort("b"), key="b")

row(df3, df4)

a,b
i64,i64
1,4
2,5
3,6
4,3
5,2
6,7
a,b
i64,i64
5,2
